# Day 09. Exercise 04
# Pipelines and OOP

## 0. Imports

In [1]:
import pandas as pd
import numpy as np
import joblib
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV

## 1. Preprocessing pipeline

Create three custom transformers, the first two out of which will be used within a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

1. `FeatureExtractor()` class:
 - Takes a dataframe with `uid`, `labname`, `numTrials`, `timestamp` from the file [`checker_submits.csv`](https://drive.google.com/file/d/14voc4fNJZiLEFaZyd8nEG-lQt5JjatYw/view?usp=sharing).
 - Extracts `hour` from `timestamp`.
 - Extracts `weekday` from `timestamp` (numbers).
 - Drops the `timestamp` column.
 - Returns the new dataframe.


2. `MyOneHotEncoder()` class:
 - Takes the dataframe from the result of the previous transformation and the name of the target column.
 - Identifies all the categorical features and transforms them with `OneHotEncoder()`. If the target column is categorical too, then the transformation should not apply to it.
 - Drops the initial categorical features.
 - Returns the dataframe with the features and the series with the target column.


3. `TrainValidationTest()` class:
 - Takes `X` and `y`.
 - Returns `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` (`test_size=0.2`, `random_state=21`, `stratified`).


In [2]:
class FeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["timestamp"] = pd.to_datetime(X["timestamp"])
        X["hour"] = X["timestamp"].dt.hour
        X["weekday"] = X["timestamp"].dt.weekday
        X = X.drop(columns=["timestamp"])
        return X

In [3]:
class MyOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, target_column):
        self.target_column = target_column
        self.encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
        self.categorical_features = None

    def fit(self, X, y=None):
        X = X.copy()
        self.categorical_features = [
            col for col in X.columns
            if X[col].dtype == "object" and col != self.target_column
        ]
        self.encoder.fit(X[self.categorical_features])
        return self
    
    def transform(self, X):
        X = X.copy()
        encoded = self.encoder.transform(X[self.categorical_features])
        encoded_df = pd.DataFrame(
            encoded,
            columns=self.encoder.get_feature_names(self.categorical_features),
            index=X.index
        )
        X = X.drop(columns=self.categorical_features)
        X = pd.concat([X, encoded_df], axis=1)
        y = X[self.target_column]
        X = X.drop(columns=[self.target_column])
        return X, y

In [4]:
class TrainValidationTest:
    def __init__(self, test_size=0.2, random_state=21):
        self.test_size = test_size
        self.random_state = random_state

    def split(self, X, y):
        X_trainval, X_test, y_trainval, y_test = train_test_split(
            X, y,
            test_size=self.test_size,
            random_state=self.random_state,
            stratify=y
        )
        X_train, X_valid, y_train, y_valid = train_test_split(
            X_trainval, y_trainval,
            test_size=self.test_size,  
            random_state=self.random_state,
            stratify=y_trainval
        )
        return X_train, X_valid, X_test, y_train, y_valid, y_test

## 2. Model selection pipeline

`ModelSelection()` class

 - Takes a list of `GridSearchCV` instances and a dict where the keys are the indexes from that list and the values are the names of the models, the example is below in the reverse order (from high-level to low-level perspective):

```
ModelSelection(grids, grid_dict)

grids = [gs_svm, gs_tree, gs_rf]

gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=jobs), where jobs you can specify by yourself

svm_params = [{'kernel':('linear', 'rbf', 'sigmoid'), 'C':[0.01, 0.1, 1, 1.5, 5, 10], 'gamma': ['scale', 'auto'], 'class_weight':('balanced', None), 'random_state':[21], 'probability':[True]}]
```

 - Method `choose()` takes `X_train`, `y_train`, `X_valid`, `y_valid` and returns the name of the best classifier among all the models on the validation set
 - Method `best_results()` returns a dataframe with the columns `model`, `params`, `valid_score` where the rows are the best models within each class of models.

```
model	params	valid_score
0	SVM	{'C': 10, 'class_weight': None, 'gamma': 'auto...	0.877778
1	Decision Tree	{'class_weight': 'balanced', 'criterion': 'gin...	0.866667
2	Random Forest	{'class_weight': None, 'criterion': 'entropy',...	0.907407
```

 - When you iterate through the parameters of a model class, print the name of that class and show the progress using `tqdm.notebook`, in the end of the cycle print the best model of that class.

```
Estimator: SVM
100%
125/125 [01:32<00:00, 1.36it/s]
Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.773
Validation set accuracy score for best params: 0.878 

Estimator: Decision Tree
100%
57/57 [01:07<00:00, 1.22it/s]
Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}
Best training accuracy: 0.801
Validation set accuracy score for best params: 0.867 

Estimator: Random Forest
100%
284/284 [06:47<00:00, 1.13s/it]
Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.855
Validation set accuracy score for best params: 0.907 

Classifier with best validation set accuracy: Random Forest
```

In [5]:
class ModelSelection:
    def __init__(self, grids, grid_dict):
        """
        grids: list of GridSearchCV objects
        grid_dict: dict {index: model_name}
        """
        self.grids = grids
        self.grid_dict = grid_dict
        self.results = []

    def choose(self, X_train, y_train, X_valid, y_valid):
        best_model_name = None
        best_valid_score = -1

        for idx, grid in enumerate(self.grids):
            model_name = self.grid_dict[idx]
            print(f"\nEstimator: {model_name}")

            with tqdm(total=len(grid.param_grid[0])) as pbar:
                grid.fit(X_train, y_train)
                pbar.update(len(grid.cv_results_["params"]))

            best_params = grid.best_params_
            best_train_score = grid.best_score_
            
            y_pred_valid = grid.best_estimator_.predict(X_valid)
            valid_score = accuracy_score(y_valid, y_pred_valid)

            print(f"Best params: {best_params}")
            print(f"Best training accuracy: {best_train_score:.3f}")
            print(f"Validation set accuracy score for best params: {valid_score:.3f}\n")

            self.results.append({
                "model": model_name,
                "params": best_params,
                "valid_score": valid_score
            })

            if valid_score > best_valid_score:
                best_valid_score = valid_score
                best_model_name = model_name

        print(f"Classifier with best validation set accuracy: {best_model_name}")
        return best_model_name

    def best_results(self):
        return pd.DataFrame(self.results)

svm_params = [{
    'kernel': ('linear', 'rbf', 'sigmoid'),
    'C': [0.01, 0.1, 1, 1.5, 5, 10],
    'gamma': ['scale', 'auto'],
    'class_weight': ('balanced', None),
    'random_state': [21],
    'probability': [True]
}]

tree_params = [{
    'criterion': ['gini', 'entropy'],
    'max_depth': np.arange(1, 50),
    'class_weight': ('balanced', None),
    'random_state': [21]
}]

rf_params = [{
    'criterion': ['gini', 'entropy'],
    'max_depth': np.arange(1, 50),
    'n_estimators': [5, 10, 50, 100],
    'class_weight': ('balanced', None),
    'random_state': [21]
}]

svm = SVC()
tree = DecisionTreeClassifier()
rf = RandomForestClassifier()

gs_svm = GridSearchCV(svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=-1)
gs_tree = GridSearchCV(tree, param_grid=tree_params, scoring='accuracy', cv=2, n_jobs=-1)
gs_rf = GridSearchCV(rf, param_grid=rf_params, scoring='accuracy', cv=2, n_jobs=-1)

grids = [gs_svm, gs_tree, gs_rf]
grid_dict = {0: "SVM", 1: "Decision Tree", 2: "Random Forest"}



## 3. Finalization

`Finalize()` class
 - Takes an estimator.
 - Method `final_score()` takes `X_train`, `y_train`, `X_test`, `y_test` and returns the accuracy of the model as in the example below:
```
final.final_score(X_train, y_train, X_test, y_test)
Accuracy of the final model is 0.908284023668639
```
 - Method `save_model()` takes a path, saves the model to this path and prints that the model was successfully saved.

In [6]:
class Finalize:
    def __init__(self, estimator):
        """
        estimator: не обязательно обученная модель
        """
        self.estimator = estimator

    def final_score(self, X_train, y_train, X_test, y_test):
        self.estimator.fit(X_train, y_train)

        y_pred = self.estimator.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        print(f"Accuracy of the final model is {acc}")
        return acc

    def save_model(self, path):
        joblib.dump(self.estimator, path)
        print(f"Model was successfully saved to {path}")

## 4. Main program

1. Load the data from the file (****name of file****).
2. Create the preprocessing pipeline that consists of two custom transformers: `FeatureExtractor()` and `MyOneHotEncoder()`:
```
preprocessing = Pipeline([('feature_extractor', FeatureExtractor()), ('onehot_encoder', MyOneHotEncoder('dayofweek'))])
```
3. Use that pipeline and its method `fit_transform()` on the initial dataset.
```
data = preprocessing.fit_transform(df)
```
4. Get `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` using `TrainValidationTest()` and the result of the pipeline.
5. Create an instance of `ModelSelection()`, use the method `choose()` applying it to the models that you want and parameters that you want, get the dataframe of the best results.
6. create an instance of `Finalize()` with your best model, use method `final_score()` and save the model in the format: `name_of_the_model_{accuracy on test dataset}.sav`.

That is it, congrats!

In [7]:
df = pd.read_csv("../data/checker_submits.csv", parse_dates=['timestamp'])

In [8]:
preprocessing = Pipeline([
    ('feature_extractor', FeatureExtractor()),
    ('onehot_encoder', MyOneHotEncoder('weekday'))
])

In [9]:
X, y = preprocessing.fit_transform(df)

In [10]:
splitter = TrainValidationTest()
X_train, X_valid, X_test, y_train, y_valid, y_test = splitter.split(X, y)

In [11]:
selector = ModelSelection(grids, grid_dict)
best_model_name = selector.choose(X_train, y_train, X_valid, y_valid)
results_df = selector.best_results()
print(results_df)


Estimator: SVM


  0%|          | 0/6 [00:00<?, ?it/s]

Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.773
Validation set accuracy score for best params: 0.878


Estimator: Decision Tree


  0%|          | 0/4 [00:00<?, ?it/s]

Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}
Best training accuracy: 0.801
Validation set accuracy score for best params: 0.867


Estimator: Random Forest


  0%|          | 0/5 [00:00<?, ?it/s]

Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.855
Validation set accuracy score for best params: 0.907

Classifier with best validation set accuracy: Random Forest
           model                                             params  \
0            SVM  {'C': 10, 'class_weight': None, 'gamma': 'auto...   
1  Decision Tree  {'class_weight': 'balanced', 'criterion': 'gin...   
2  Random Forest  {'class_weight': None, 'criterion': 'entropy',...   

   valid_score  
0     0.877778  
1     0.866667  
2     0.907407  


In [12]:
best_index = list(grid_dict.keys())[list(grid_dict.values()).index(best_model_name)]
final_model = grids[best_index].best_estimator_  

In [13]:
final = Finalize(final_model)
acc = final.final_score(X_train, y_train, X_test, y_test)

Accuracy of the final model is 0.9053254437869822


In [14]:
filename = f"{best_model_name}_{acc:.3f}.sav"
final.save_model(filename)

Model was successfully saved to Random Forest_0.905.sav
